In [1]:
import pandas as pd
import numpy as np
import math
import os
import neurokit2 as nk
import nolds
from scipy import signal
import heartpy as hp

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
where_PPG = 'C:/Users/citioplab/聯發科PPG Raw Data'

# 前處理 acc

In [5]:
# 消除基線飄移
def filt_ma(acc):
    newlist = [x for x in acc if math.isnan(x) == False]
    b, a = signal.butter(3, 0.1, btype="highpass", output="ba")
    ma_acc = signal.filtfilt(b, a, newlist)
    return ma_acc

# 讀所有病人所有的資料

In [6]:
def time_imputation(df, column, filename):
    # 向下補值
    df[column].replace(0, np.nan, inplace=True)
    df[column].fillna(method='ffill', inplace=True)
    # 補開頭
    df[column].replace(np.nan, filename, inplace=True)
    # 統一格式
    df[column] = df[column].astype(str).str.rstrip('.0')
    df[column] = df[column].str.pad(width=14, side='right', fillchar='0')
    df[column] = pd.to_datetime(df[column], format='%Y%m%d%H%M%S', errors='coerce')
    return df

In [7]:
# 讀取資料，並做成dataframe
def read_data(where_PPG, patient, verbose):

    day_dict = {}

    if "(x)" not in patient:
        goto = where_PPG + "//" + str(patient) + "//"
        raw =  os.listdir(goto)
        
        for raw_data in raw:
            new_df = pd.DataFrame(columns=['Measure_time', 'X_acc', 'Y_acc', 'Z_acc'])

            if "raw" in raw_data.split(".")[0]: 
                filename = raw_data.split(".")[0].replace('_raw.xlsx', '')[:14]
                df = pd.read_csv(goto + raw_data ,header = None)
                df = df.dropna()

                # 取出csv中的欄位值
                measure_time = np.array(df[0].values.tolist()).T[0]
                x_acc = filt_ma(np.array(df[[4]].values.tolist()).T[0]) # 消除基線飄移
                y_acc = filt_ma(np.array(df[[5]].values.tolist()).T[0]) # 消除基線飄移
                z_acc = filt_ma(np.array(df[[6]].values.tolist()).T[0]) # 消除基線飄移

                # 做成dataframe
                part_df = pd.DataFrame({'Measure_time': df[0].tolist(), 'X_acc':x_acc, 'Y_acc': y_acc, 'Z_acc': z_acc})
                file_start_time = pd.to_datetime(filename[:14], format='%Y%m%d%H%M%S', errors='coerce') - pd.Timedelta(seconds=1)
                part_df = time_imputation(part_df, 'Measure_time', file_start_time)
                
                day_dict[filename] = part_df
        if verbose:
            print(f"{patient} read, length = {day_dict.__len__()}")
    else:
        print(f"{patient} empty")
    return day_dict
        

In [8]:
# # 確認每秒不超過64筆資料
# for data in day_dict.values():
#     period_counts_df = pd.DataFrame({'Measure_time': data['Measure_time'].value_counts().index, 'Count': data['Measure_time'].value_counts().values})
#     sorted_counts_df = period_counts_df.sort_values(by='Measure_time').reset_index(drop=True)
#     if len(sorted_counts_df.loc[sorted_counts_df['Count'] > 64, 'Measure_time']) > 0:
#         print(sorted_counts_df.loc[sorted_counts_df['Count'] > 64, 'Measure_time'])

# 計算SPT

In [9]:
# def percentile(data, perc: int):
#     size = len(data)
#     return sorted(data)[int(math.ceil((size * perc) / 100)) - 1]

In [10]:
def percentile(data, perc: int):
    size = len(data)
    if size == 0:
        return None  # Handle empty list case
    sorted_data = sorted(data)
    index = int(math.ceil((size * perc) / 100)) - 1
    index = min(index, size - 1)  # Ensure index is within range
    return sorted_data[index]

In [11]:
def find_threshold(day_dict, q):
    angle_diff = []
    for data in day_dict.values():
        angle_diff.extend(data['rolling_median_angle'].values.tolist())
    # print(len(angle_diff))
    threshold = percentile(angle_diff,q)
    return threshold

In [12]:
def five_sec_acc_rolling_med(data):
    # 取平均改以秒為單位
    X_avg = pd.DataFrame(data.groupby('Measure_time')['X_acc'].mean())
    Y_avg = pd.DataFrame(data.groupby('Measure_time')['Y_acc'].mean())
    Z_avg = pd.DataFrame(data.groupby('Measure_time')['Z_acc'].mean())
    df_sec = pd.DataFrame({'X_acc': X_avg['X_acc'], 'Y_acc': Y_avg['Y_acc'], 'Z_acc': Z_avg['Z_acc']})
    
    # Compute rolling medians for x, y, z coordinates
    rolling_medians_x = df_sec['X_acc'].rolling(window=5).median()
    rolling_medians_y = df_sec['Y_acc'].rolling(window=5).median()
    rolling_medians_z = df_sec['Z_acc'].rolling(window=5).median()

    # Combine rolling medians into a new DataFrame
    rolling_medians_df = pd.DataFrame({
        'rolling_median_x': rolling_medians_x,
        'rolling_median_y': rolling_medians_y,
        'rolling_median_z': rolling_medians_z
    })
    rolling_medians_df = rolling_medians_df.dropna()

    return rolling_medians_df

In [13]:
def calculate_angle(acc_x, acc_y, acc_z):
    deg = math.atan(acc_x / (math.sqrt(acc_y**2 + acc_z**2)))*180/math.pi
    return deg

In [14]:
def angle_diff(rolling_medians_df):
    rolling_medians_df['angle'] = rolling_medians_df.apply(lambda row: calculate_angle(row['rolling_median_x'], row['rolling_median_y'], row['rolling_median_z']), axis=1)
    
    # Compute 5 sec rolling average for angles
    rolling_avg_angle = pd.DataFrame(rolling_medians_df['angle'].rolling(window=5).mean())
    # rolling_avg_angle = rolling_avg_angle.dropna()

    # Compute the difference between consecutive angle averages
    rolling_avg_angle['angle_diff'] = rolling_avg_angle['angle'].diff()
    rolling_avg_angle['angle_diff'] = abs(rolling_avg_angle['angle_diff'])
    rolling_avg_angle = rolling_avg_angle.dropna()

    # 確認測量時間是連續的(無中斷，2019-04-13 15:36:29是中斷的)
    rolling_avg_angle = rolling_avg_angle.reset_index()
    rolling_avg_angle['time_diff'] = pd.to_timedelta(rolling_avg_angle['Measure_time'].diff())
    filtered_df = rolling_avg_angle[rolling_avg_angle['time_diff'] != pd.Timedelta('0 days 00:00:01')]
    # if filtered_df.shape[0] > 1:
    #     return "Time is not continuous"
    
    # Compute the 5 minutes rolling median for the angle differences
    rolling_median_5min_angle = pd.DataFrame({
        'Measure_time': rolling_avg_angle['Measure_time'],
        'rolling_median_angle': rolling_avg_angle['angle_diff'].rolling(window=300).median()
    })
    rolling_median_5min_angle = rolling_median_5min_angle.dropna()
    
    return rolling_median_5min_angle


In [15]:
def block_detection(angDiff, threshold, verbose):
    # 標註靜止的時間
    angDiff['still'] = angDiff['rolling_median_angle'].apply(lambda x: 1 if x < threshold else 0)  
    if verbose:
        print(angDiff['still'].value_counts())
        
    # 找到所有的blocks
    block_start = []
    block_end = []
    pre_value = 0

    for idx, value in enumerate(angDiff['still']):
        if pre_value == 0 and value == 1:
            block_start.append(angDiff['Measure_time'].iloc[idx])
        elif pre_value == 1 and value == 0:
            block_end.append(angDiff['Measure_time'].iloc[idx-1])
        pre_value = value
    block_duration = [block_end[i] - block_start[i] for i in range(len(block_end))]
    if verbose:
        print(len(block_start), len(block_end), len(block_duration))
    blocks_df = pd.DataFrame({'block_start': block_start[:len(block_end)], 'block_end': block_end, 'duration':block_duration})
    
    # 留下持續超過30分鐘的blocks，並計算time gap
    spt_blocks = blocks_df[blocks_df['duration'] > pd.Timedelta('0 days 00:10:00')]
    time_gap_list = []
    for idx, value in enumerate(spt_blocks['block_end']):
        if idx < len(spt_blocks)-1:
            time_gap_list.append(spt_blocks['block_start'].iloc[idx+1] - spt_blocks['block_end'].iloc[idx])
        else:
            time_gap_list.append(np.nan)
    spt_blocks['time_gap'] = time_gap_list
    return spt_blocks

In [16]:
def turn_blocks_to_SPT(spt_blocks):
    # 整併間隔<60min的blocks
    SPT_start = 0
    SPT_end = 0
    SPT_duration = pd.Timedelta('0 days 00:00:00')
    temp_start = 0
    temp_end = 0
    temp_duration = pd.Timedelta('0 days 00:00:00')

    for idx, value in enumerate(spt_blocks['time_gap']):

        if value < pd.Timedelta('0 days 00:60:00'):
            if idx == 0:
                temp_start = spt_blocks['block_start'].iloc[idx]
            elif spt_blocks['time_gap'].iloc[idx-1] > pd.Timedelta('0 days 00:60:00'):
                temp_start = spt_blocks['block_start'].iloc[idx]

            temp_end = spt_blocks['block_end'].iloc[idx+1]
            temp_duration += spt_blocks['duration'].iloc[idx]

        else:
            if temp_duration > SPT_duration:
                SPT_duration = temp_duration
                SPT_start = temp_start
                SPT_end = temp_end
            temp_duration = pd.Timedelta('0 days 00:00:00')
            
    return SPT_start, SPT_end


# MAIN

In [18]:
# 所有病患
patients = os.listdir(where_PPG)
    
for patient in patients:
    by_patient = pd.DataFrame(columns=['Date_start', 'Date_end', 'SPT_start', 'SPT_end']) 

    try:
        all_data = read_data(where_PPG, patient, verbose=False)        
            
        for date in all_data.keys(): 
            day_data = all_data[date]
            
            rolling_medians_df = five_sec_acc_rolling_med(day_data) # step1
            angDiff = angle_diff(rolling_medians_df) #step2-5
            all_data[date] = angDiff #把raw data(acc)改存成angle_diff
            
        threshold = find_threshold(all_data, q=40) #step 6
        
        for date in all_data.keys():            
            spt_blocks = block_detection(all_data[date], threshold, verbose=False) #step7
            SPT_start, SPT_end = turn_blocks_to_SPT(spt_blocks) #step 8-10
            
            new_row = {'Date_start': pd.to_datetime(date[:14]), 'Date_end': pd.to_datetime(all_data[date]['Measure_time'].iloc[-1]), 'SPT_start': SPT_start, 'SPT_end': SPT_end}
            by_patient.loc[len(by_patient.index)] = new_row

        if by_patient.shape[0] > 0: #沒有任何blocks
            by_patient.to_csv(f'C:/Users/citioplab/Desktop/github/Lab/PPG/SPT windows new/{patient}_SPT.csv', mode='w', header=True, index=False)  
            print(f"{patient} done, length = {by_patient.shape[0]}") 
            
    except Exception as e:
        if all_data.__len__() > 0: #病人本來有資料
            print(f"{patient} is not available, {e}")

TVGH001 (x) empty
TVGH002 done, length = 9
TVGH003 done, length = 17
TVGH004 (x) empty
TVGH005 is not available, '<' not supported between instances of 'float' and 'Timedelta'
TVGH006 done, length = 32
TVGH007 done, length = 7
TVGH008 (x) empty
TVGH009 done, length = 12
TVGH010 (x) empty
TVGH011 (x) empty
TVGH012 done, length = 30
TVGH013 done, length = 13
TVGH014 done, length = 12
TVGH015 (x) empty
TVGH016 done, length = 14
TVGH017 done, length = 9
TVGH018 done, length = 18
TVGH019 done, length = 17
TVGH020 done, length = 10
TVGH021 done, length = 9
TVGH022 done, length = 19
TVGH023 done, length = 12
TVGH024 done, length = 13
TVGH025 done, length = 18
TVGH026 is not available, '<' not supported between instances of 'float' and 'Timedelta'
TVGH027 done, length = 9
TVGH028 done, length = 11
TVGH029 done, length = 16
TVGH030 (x) empty
TVGH031 done, length = 20
TVGH032 done, length = 11
TVGH033 done, length = 17
TVGH034 done, length = 9
TVGH035 (x) empty
TVGH036 (x) empty
TVGH037 done, le

In [19]:
# # 所有病患
# patients = os.listdir(where_PPG)
    
# for patient in patients:
#     by_patient = pd.DataFrame(columns=['Date_start', 'Date_end', 'SPT_start', 'SPT_end']) 

#     all_data = read_data(where_PPG, patient, verbose=False)        
        
#     for date in all_data.keys(): 
#         day_data = all_data[date]
        
#         rolling_medians_df = five_sec_acc_rolling_med(day_data) # step1
#         angDiff = angle_diff(rolling_medians_df) #step2-5
#         all_data[date] = angDiff #把raw data(acc)改存成angle_diff
        
#     threshold = find_threshold(all_data, q=45) #step 6
    
#     for date in all_data.keys():            
#         spt_blocks = block_detection(all_data[date], threshold, verbose=False) #step7
#         SPT_start, SPT_end = turn_blocks_to_SPT(spt_blocks) #step 8-10
        
#         new_row = {'Date_start': pd.to_datetime(date[:14]), 'Date_end': pd.to_datetime(all_data[date]['Measure_time'].iloc[-1]), 'SPT_start': SPT_start, 'SPT_end': SPT_end}
#         by_patient.loc[len(by_patient.index)] = new_row

#     if by_patient.shape[0] > 0: #沒有任何blocks
#         by_patient.to_csv(f'C:/Users/citioplab/Desktop/github/Lab/PPG/SPT windows new/{patient}_SPT.csv', mode='w', header=True, index=False)  
#         print(f"{patient} done, length = {by_patient.shape[0]}") 
        
 

In [20]:
spt_blocks

,block_start,block_end,duration,time_gap
2,2021-03-04 21:39:16,2021-03-04 21:53:52,0 days 00:14:36,0 days 00:13:59
13,2021-03-04 22:07:51,2021-03-04 22:23:02,0 days 00:15:11,0 days 00:03:07
15,2021-03-04 22:26:09,2021-03-04 22:40:00,0 days 00:13:51,0 days 00:40:33
32,2021-03-04 23:20:33,2021-03-04 23:44:23,0 days 00:23:50,0 days 02:07:10
56,2021-03-05 01:51:33,2021-03-05 02:03:36,0 days 00:12:03,0 days 00:02:28
64,2021-03-05 02:06:04,2021-03-05 02:16:26,0 days 00:10:22,0 days 04:41:36
113,2021-03-05 06:58:02,2021-03-05 07:13:09,0 days 00:15:07,0 days 00:52:37
128,2021-03-05 08:05:46,2021-03-05 08:15:58,0 days 00:10:12,NaT
